In [3]:
import pandas as pd    

full_merged_df = pd.read_csv('test_vel.csv', parse_dates=["datetime"])  # Ensure 'datetime' is parsed correctly

full_merged_df.head()

# GCL = velocity
# DGL = gate closed/open
# FP2VMa = scenario for height of the water level
# Modeled_Historic = historic modeling for water level 
# in_op = gate in operation; 1 = in operation


,GLC,datetime,Velocity_Category,consecutive_groups,min_datetime,max_datetime,date,count,streak_duration,DGL,gate_min_datetime,gate_max_datetime,gate_count,gate_streak_duration,time_unit,FP2VMa,in_op,Modeled_Historic,week
0,0.111414,2021-05-01 00:00:00,Under 8ft/s,1,2021-05-01 00:00:00,2021-05-01 01:30:00,2021-05-01,7,1.75,Open,2021-05-01 00:00:00,2021-05-01 00:15:00,2,0.50,0.25,5.044312,1,5.047670,17
1,0.314615,2021-05-01 00:15:00,Under 8ft/s,1,2021-05-01 00:00:00,2021-05-01 01:30:00,2021-05-01,7,1.75,Open,2021-05-01 00:00:00,2021-05-01 00:15:00,2,0.50,0.25,4.940485,1,4.943711,17
2,5.152236,2021-05-01 00:30:00,Under 8ft/s,1,2021-05-01 00:00:00,2021-05-01 01:30:00,2021-05-01,7,1.75,Closed,2021-05-01 00:30:00,2021-05-01 04:30:00,17,4.25,0.25,4.826855,1,4.819327,17
3,6.296503,2021-05-01 00:45:00,Under 8ft/s,1,2021-05-01 00:00:00,2021-05-01 01:30:00,2021-05-01,7,1.75,Closed,2021-05-01 00:30:00,2021-05-01 04:30:00,17,4.25,0.25,4.876334,1,4.704493,17
4,6.569984,2021-05-01 01:00:00,Under 8ft/s,1,2021-05-01 00:00:00,2021-05-01 01:30:00,2021-05-01,7,1.75,Closed,2021-05-01 00:30:00,2021-05-01 04:30:00,17,4.25,0.25,4.893825,1,4.606611,17
